# Option 1. Predict profit per user per day

## Description

We have LTV values accumulated from day to day. The idea for this option is to calculate profit per day using formula profit_N = LTV_N - LTV_N-1, then extrapolate this profit till 60th day, then calculate sum for 60 days, make agregation 

In [18]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# zero LTVs are excluded from LTVs vector
def get_ltvs_list_delta(df):
    ltvs = []

    if df.Ltv1 > 0:
        ltvs.append(df.Ltv1)

    if df.Ltv2 > 0:
        ltvs.append(df.Ltv2 - df.Ltv1)

    if df.Ltv3 > 0:
        ltvs.append(df.Ltv3 - df.Ltv2)
    
    if df.Ltv4 > 0:
        ltvs.append(df.Ltv4 - df.Ltv3)
    
    if df.Ltv5 > 0:
        ltvs.append(df.Ltv5 - df.Ltv4)
    
    if df.Ltv6 > 0:
        ltvs.append(df.Ltv6 - df.Ltv5)
    
    if df.Ltv7 > 0:
        ltvs.append(df.Ltv7 - df.Ltv6)  

    
    return ltvs

def get_ltvs_list_delta_all_users(df):
    ltvs = []

    if df.Ltv1 > 0:
        ltvs.append(df.Ltv1 * df.Users)

    if df.Ltv2 > 0:
        ltvs.append((df.Ltv2 - df.Ltv1)* df.Users)

    if df.Ltv3 > 0:
        ltvs.append((df.Ltv3 - df.Ltv2)* df.Users)
    
    if df.Ltv4 > 0:
        ltvs.append((df.Ltv4 - df.Ltv3)* df.Users)
    
    if df.Ltv5 > 0:
        ltvs.append((df.Ltv5 - df.Ltv4)* df.Users)
    
    if df.Ltv6 > 0:
        ltvs.append((df.Ltv6 - df.Ltv5)* df.Users)
    
    if df.Ltv7 > 0:
        ltvs.append((df.Ltv7 - df.Ltv6)* df.Users)  

    
    return ltvs
def checkLessZero(v):
    if v >= 0:
        return v
    return 0

def linearRegressionDaily(y):
    X = []
    for i in range(0, len(y)) :
        X.append([i])

    model = LinearRegression()
    model.fit(X, y)   

    x = []
    for i in range(len(y), 59):
        x.append([i])

    yNew = model.predict(x)

    yNew = np.array(list(map(checkLessZero, yNew)))
    return yNew.sum() + sum(y)

### CSV file

In [13]:
data = pd.read_csv("../assets/data/test_data.csv")
data['non_zero_profits'] = data.apply(get_ltvs_list_delta, axis=1)

data['profit_60'] = data['non_zero_profits'].apply(linearRegressionDaily)
data

,UserId,CampaignId,Country,Ltv1,Ltv2,Ltv3,Ltv4,Ltv5,Ltv6,Ltv7,non_zero_profits,profit_60
0,1,81855ad8-681d-4d86-91e9-1e00167939cb,TR,1.549970,2.252664,2.298636,2.884009,3.696002,5.714437,0.000000,"[1.5499697874482206, 0.7026938182501425, 0.045...",199.570368
1,2,6325253f-ec73-4dd7-a9e2-8bf921119c16,DE,0.475155,0.615663,0.916867,1.462596,0.000000,0.000000,0.000000,"[0.4751548999010757, 0.14050841572875866, 0.30...",81.997672
2,3,6694d2c4-22ac-4208-a007-2939487f6999,FR,1.683877,2.115496,2.581127,3.007737,4.064288,5.823792,9.420708,"[1.6838771724439194, 0.43161920528463904, 0.46...",571.696978
3,4,eb9d18a4-4784-445d-87f3-c67cf22746e9,TW,0.031391,0.034643,0.036539,0.045443,0.073090,0.086438,0.000000,"[0.0313905693506446, 0.0032524078455933025, 0....",0.406897
4,5,0f070244-8615-4bda-8831-3f6a8eb668d2,TW,1.741325,1.793922,1.855427,0.000000,0.000000,0.000000,0.000000,"[1.7413250766426165, 0.05259734587821008, 0.06...",1.855427
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,680b4e7c-8b76-4a1b-9d49-d4955c848621,US,0.990175,1.036283,1.415886,1.979443,2.319511,0.000000,0.000000,"[0.990175317868198, 0.04610786625128882, 0.379...",2.771898
9996,9997,6325253f-ec73-4dd7-a9e2-8bf921119c16,US,0.504046,0.558039,0.887269,1.188758,0.000000,0.000000,0.000000,"[0.5040458162984021, 0.0539928474475968, 0.329...",1.989214
9997,9998,5fb90bad-b37c-4821-b6d9-5526a41a9504,FR,1.416509,1.445417,1.494038,1.552371,0.000000,0.000000,0.000000,"[1.4165089569461125, 0.028908346060124712, 0.0...",1.552371
9998,9999,6694d2c4-22ac-4208-a007-2939487f6999,US,1.897832,3.141638,4.133432,5.745912,0.000000,0.000000,0.000000,"[1.8978319536360493, 1.2438060144037872, 0.991...",12.405610


In [14]:
groupCountry = data.groupby('Country')
groupCountry['profit_60'].sum()/groupCountry.size()

Country
CA    45.754939
DE    47.016006
FR    50.503740
GB    49.121313
IT    46.508203
JP    47.659153
TR    49.825542
TW    46.441726
US    52.645223
dtype: float64

### JSON file

Here the estimation will be dane based on profit for all users in row

In [19]:
data = pd.read_json("../assets/data/test_data.json")
data['non_zero_profits'] = data.apply(get_ltvs_list_delta_all_users, axis=1)

data['profit_60'] = data['non_zero_profits'].apply(linearRegressionDaily)
data

,CampaignId,Country,Ltv1,Ltv2,Ltv3,Ltv4,Ltv5,Ltv6,Ltv7,Users,non_zero_profits,profit_60
0,9566c74d-1003-4c4d-bbbb-0407d1e2c649,TR,1.954250,1.994133,3.012637,3.113805,3.201461,3.796799,4.321961,93,"[181.74527678761788, 3.7090872813800093, 94.72...",404.795617
1,6694d2c4-22ac-4208-a007-2939487f6999,IT,0.464016,0.708067,0.947904,1.385559,1.812879,2.423993,3.393102,97,"[45.00958337528079, 23.672872538829118, 23.264...",15724.240856
2,9566c74d-1003-4c4d-bbbb-0407d1e2c649,DE,0.623120,0.799067,0.986064,1.139339,1.715935,2.221786,2.974998,101,"[62.935132484112884, 17.77067979880444, 18.886...",10498.856593
3,5fb90bad-b37c-4821-b6d9-5526a41a9504,TR,1.891967,3.042047,3.465635,3.845075,5.800936,9.630334,10.730099,116,"[219.46820997649692, 133.40929152461064, 49.13...",39180.001712
4,6694d2c4-22ac-4208-a007-2939487f6999,FR,0.542177,0.606477,0.696125,1.107782,1.550166,1.655706,2.556096,109,"[59.09728724576607, 7.008712720129529, 9.77160...",11364.635679
...,...,...,...,...,...,...,...,...,...,...,...,...
85,81855ad8-681d-4d86-91e9-1e00167939cb,IT,0.320981,0.339258,0.529164,0.725839,0.836077,0.932567,1.120011,119,"[38.19673986387336, 2.175000742041787, 22.5987...",206.690466
86,5fb90bad-b37c-4821-b6d9-5526a41a9504,GB,0.104668,0.132496,0.191864,0.236123,0.284503,0.354849,0.517288,108,"[11.304187605507709, 3.005432855240005, 6.4117...",1934.460766
87,6694d2c4-22ac-4208-a007-2939487f6999,US,0.242256,0.250453,0.255565,0.341572,0.443686,0.509863,0.676017,213,"[51.60058234173939, 1.745968819113492, 1.08870...",1034.595728
88,95af5a25-3679-41ba-a2ff-6cd471c483f1,FR,1.565566,2.082341,2.804472,3.374082,4.120237,6.216649,7.795717,104,"[162.8188914628176, 53.744597853285654, 75.101...",25201.813577


In [20]:
groupCountry = data.groupby('Country')
groupCountry['profit_60'].sum()/groupCountry['Users'].sum()

Country
CA     88.750416
DE     91.583788
FR    207.308624
GB    150.341928
IT     59.798670
JP     50.664659
TR     83.889447
TW    129.744127
US    106.134043
dtype: float64